In [2]:
import pandas as pd

# Define the file path
nodes_path = "Data/Nodes_processed.csv"
roads_path = "Data/Roads_processed.csv"
bridges_path = "Data/Bridges_processed.csv"
bridges_selected_path = "Data/Bridges_selected.csv"


# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_bridges_sel = pd.read_csv(
    bridges_selected_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_4452\2470735124.py:11: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_4452\2470735124.py:12: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [3]:
# df_nodes.columns, df_roads.columns, df_bridges_all.columns, df_bridges_sel.columns

In [4]:
# Specify the columns to include in the new DataFrame
columns_to_include = ['ID_mosta', 'Zemepisná_dĺžka',
                      'Zemepisná_šírka', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID']

# Create the new DataFrame df_routes by selecting the specified columns from df_bridges
df_routes = df_bridges_all[columns_to_include].copy()

# Drop rows where the condition df_bridges["n_2023"] == 0 is met
df_routes = df_routes[df_bridges_all["n_2023"] != 0]


# Change the data types of columns in df_routes
df_routes['ID_mosta'] = df_routes['ID_mosta'].astype(str)
df_routes['Zemepisná_dĺžka'] = df_routes['Zemepisná_dĺžka'].astype(float)
df_routes['Zemepisná_šírka'] = df_routes['Zemepisná_šírka'].astype(float)
df_routes['Počiatočný_uzol'] = df_routes['Počiatočný_uzol'].astype(str)
df_routes['Koncový_uzol'] = df_routes['Koncový_uzol'].astype(str)
df_routes['úsek_ID'] = df_routes['úsek_ID'].astype(int)

In [5]:
df_routes.columns, df_bridges_sel.columns

(Index(['ID_mosta', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Počiatočný_uzol',
        'Koncový_uzol', 'úsek_ID'],
       dtype='object'),
 Index(['ID_Mosta', 'ID_úseku', 'Start', 'End', 'Okres'], dtype='object'))

In [6]:
df_routes.head()

,ID_mosta,Zemepisná_dĺžka,Zemepisná_šírka,Počiatočný_uzol,Koncový_uzol,úsek_ID
0,M5897,17.123319,48.790152,3424A11500,3424A11600,259391
1,M1979,17.079098,48.757287,3424A09200,3424A01900,259368
2,M6525,17.031675,48.684515,3424A02000,3442A04800,271
3,M4584,17.020574,48.663688,3442A27000,3442A27301,284169
4,M3053,17.001660,48.639120,3442A01113,3442A01109,6670


In [7]:
df_bridges_sel.head(), df_bridges_sel.shape

(  ID_Mosta  ID_úseku       Start         End            Okres
 0    M5833       153  3722A01600  3722A51100           Prešov
 1    M1902      3511  3722A52702  3722A01600           Prešov
 2     M121     91360  2633A42600  2633A01600           Žilina
 3    M6035     90094  2631A00222  2631A01601           Žilina
 4    M5676      5450  3831A02100  3742A03600  Košice - okolie,
 (148, 5))

In [8]:
df_routes = df_routes.merge(
    df_bridges_sel, left_on='ID_mosta', right_on='ID_Mosta', how='inner')

In [9]:
# Add columns for latitude and longitude of the start and end nodes
df_routes['StartNode_Latitude'] = df_routes['Start'].map(
    df_nodes.set_index('Číslo_uzla')['Zemepisná_šírka'])
df_routes['StartNode_Longitude'] = df_routes['Start'].map(
    df_nodes.set_index('Číslo_uzla')['Zemepisná_dĺžka'])
df_routes['EndNode_Latitude'] = df_routes['End'].map(
    df_nodes.set_index('Číslo_uzla')['Zemepisná_šírka'])
df_routes['EndNode_Longitude'] = df_routes['End'].map(
    df_nodes.set_index('Číslo_uzla')['Zemepisná_dĺžka'])

In [10]:
from math import radians, cos, sin, asin, sqrt, pi


def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6367 * c
    return km


def get_hexagon_coordinates(center_lat, center_lon, radius):
    coordinates = []
    for i in range(6):
        angle = 2 * pi / 6 * i
        dx = radius * cos(angle)
        dy = radius * sin(angle)
        point_lat = center_lat + (180/pi) * (dy / 6378137)
        point_lon = center_lon + (180/pi) * \
            (dx / 6378137) / cos(center_lat * pi/180)
        coordinates.append((point_lat, point_lon))
    return coordinates


# Radius of the circle in meters in which hexagon is inscribed
radius = 20  # 10 meters radius in kilometers

# Create a new column 'Hexagon_Coordinates' in df_routes
df_routes['Hexagon_Coordinates'] = None  # Initialize with None values

# Iterate through each row in the DataFrame
for index, row in df_routes.iterrows():
    # Get the latitude and longitude of the bridge
    latitude = row['Zemepisná_šírka']
    longitude = row['Zemepisná_dĺžka']

    # Create hexagon coordinates
    hexagon_coords = get_hexagon_coordinates(latitude, longitude, radius)
    hexagon_coords.append(hexagon_coords[0])

    # Convert hexagon coordinates from (latitude, longitude) tuples to [longitude, latitude] lists
    hexagon_coords_for_api = [[lon, lat] for lat, lon in hexagon_coords]

    # Assign the hexagon coordinates to the 'Hexagon_Coordinates' column
    df_routes.at[index, 'Hexagon_Coordinates'] = hexagon_coords

In [11]:
import json

def generate_graphhopper_request(df, row_index, profile="car", multiplier=1):
    row = df.iloc[row_index]
    hexagon_coords = row['Hexagon_Coordinates']
    # Correct the order of coordinates and convert them to list
    hexagon_coords_corrected = [[lon, lat] for lat, lon in hexagon_coords]

    data = {
        "points": [
            [row['StartNode_Longitude'], row['StartNode_Latitude']],
            [row['EndNode_Longitude'], row['EndNode_Latitude']]
        ],
        "profile": profile,
        "points_encoded": False,
        "ch.disable": True,
        "custom_model": {
            "distance_influence": 15,
            "priority": [
                {
                    "if": "in_area1",
                    "multiply_by": multiplier
                }
            ],
            "speed": [],
            "areas": {
                "area1": {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [hexagon_coords_corrected]
                    },
                    "properties": {}
                }
            }
        }
    }
    # Convert the dictionary to a JSON string
    json_data = json.dumps(data, indent=2)
    return json_data

In [12]:
pd.set_option('display.max_rows', None)
df_routes.head(5)

,ID_mosta,Zemepisná_dĺžka,Zemepisná_šírka,Počiatočný_uzol,Koncový_uzol,úsek_ID,ID_Mosta,ID_úseku,Start,End,Okres,StartNode_Latitude,StartNode_Longitude,EndNode_Latitude,EndNode_Longitude,Hexagon_Coordinates
0,M1979,17.079098,48.757287,3424A09200,3424A01900,259368,M1979,81866,3424A01900,3424A10900,Skalica,48.729199,17.048036,48.799728,17.158043,"[(48.757287384, 17.079370628365783), (48.75744..."
1,M1833,16.994971,48.634732,3442A01106,3441A02300,8719,M1833,80070,3441A00200,3442A01106,Senica,NaN,NaN,48.637892,16.999123,"[(48.634732118, 16.995242375727763), (48.63488..."
2,M3446,17.034921,48.241370,4422A33200,4422A40300,460524,M3446,80129,4422A05403,4422A32100,Bratislava IV,48.205366,17.042718,48.252092,17.033278,"[(48.241369836, 17.035190832766478), (48.24152..."
3,M7099,17.047200,48.189060,4422A45001,4422A45101,460763,M7099,81581,4422A05905,4422A05402,Bratislava IV,48.181802,17.055594,48.204278,17.043042,"[(48.18906, 17.047469491298028), (48.189215592..."
4,M5333,17.131041,48.125991,4424A02802,4424A52301,7589,M5333,84112,4424A00505,4424A50901,Bratislava V,48.129505,17.129882,48.094428,17.121275,"[(48.125990691, 17.1313097892143), (48.1261462..."


In [13]:
# Example usage
json_request = generate_graphhopper_request(
    df_routes, 50, profile="car", multiplier=1)
print(json_request)

{
  "points": [
    [
      18.88083939,
      48.7778947
    ],
    [
      18.90137031,
      48.75080705
    ]
  ],
  "profile": "car",
  "points_encoded": false,
  "ch.disable": true,
  "custom_model": {
    "distance_influence": 15,
    "priority": [
      {
        "if": "in_area1",
        "multiply_by": 1
      }
    ],
    "speed": [],
    "areas": {
      "area1": {
        "type": "Feature",
        "geometry": {
          "type": "Polygon",
          "coordinates": [
            [
              [
                18.905636006762556,
                48.765469094
              ],
              [
                18.905499721381275,
                48.765624686771325
              ],
              [
                18.90522715061872,
                48.765624686771325
              ],
              [
                18.90509086523744,
                48.765469094
              ],
              [
                18.90522715061872,
                48.76531350122867
              ]

In [14]:
rewrite_coords = [

    {
        "ID_úseku": "80640",
        "Start": "ETRS89 (LON, LAT) / GPS: 18.06722, 48.82571",
        "End": "ETRS89 (LON, LAT) / GPS: 18.17999, 48.77522"
    }

]

for item in rewrite_coords:
    # Extract ID for matching
    id_useku = item["ID_úseku"]

    # Extract and parse Start coordinates
    start_coords = item["Start"].split("GPS: ")[1].split(", ")
    start_lat, start_lon = float(start_coords[1]), float(start_coords[0])

    # Extract and parse End coordinates
    end_coords = item["End"].split("GPS: ")[1].split(", ")
    end_lat, end_lon = float(end_coords[1]), float(end_coords[0])

    # Find the index of the row with the matching ID_úseku
    indices = df_routes[df_routes['ID_úseku'] == id_useku].index

    # Update the coordinates in df_routes for all matching indices
    for idx in indices:
        df_routes.loc[idx, 'StartNode_Latitude'] = start_lat
        df_routes.loc[idx, 'StartNode_Longitude'] = start_lon
        df_routes.loc[idx, 'EndNode_Latitude'] = end_lat
        df_routes.loc[idx, 'EndNode_Longitude'] = end_lon

Koniec scriptu

In [15]:
# Change the "ID_úseku" column's data type to string
df_routes["ID_úseku"] = df_routes["ID_úseku"].astype(str)

# Replace 'your_file_path.csv' with the desired file path and name
file_path = 'Data/routes_selected.csv'

# Save the DataFrame as a CSV file
df_routes.to_csv(file_path, index=False)

In [16]:
# Change the "ID_úseku" column's data type to string
df_routes["ID_úseku"] = df_routes["ID_úseku"].astype(str)

# Replace 'your_file_path.xlsx' with the desired file path and name with .xlsx extension
file_path = 'Data/routes_selected.xlsx'

# Save the DataFrame as an Excel file
df_routes.to_excel(file_path, index=False)

In [17]:
import json


def get_text_from_json_file(file_path):
    # Open and read the JSON data from the file with UTF-8 encoding
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    # Convert the JSON data to a plain string
    plain_string = json.dumps(json_data)

    return plain_string


# Example usage
file_path = "Data/route_temp.json"
plain_string = get_text_from_json_file(file_path)

print(plain_string)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/route_temp.json'

In [18]:
import pandas as pd

# Replace 'your_file_path.xlsx' with the actual file path and name
file_path = 'Data/routes_selected2.xlsx'

# Load the Excel file into a DataFrame
df_routes_sel = pd.read_excel(file_path)


In [19]:
df_routes_sel.columns

Index(['ID_mosta', 'Zemepisná_šírka', 'Zemepisná_dĺžka', 'úsek_ID', 'ID_úseku',
       'Start', 'End', 'Okres', 'StartNode_Latitude', 'StartNode_Longitude',
       'EndNode_Latitude', 'EndNode_Longitude', 'Car_route', 'Car_alt',
       'Truck_route', 'Truck_alt', 'ok'],
      dtype='object')

In [20]:
df_routes_sel.iloc[0]["Car_route"]

'{"hints": {"visited_nodes.sum": 164, "visited_nodes.average": 164}, "info": {"copyrights": ["GraphHopper", "OpenStreetMap contributors"], "took": 3, "road_data_timestamp": "2023-12-17T05:00:00Z"}, "paths": [{"distance": 11685.516, "weight": 809.891005, "time": 634610, "transfers": 0, "points_encoded": false, "bbox": [17.048037, 48.7292, 17.158042, 48.800737], "points": {"type": "LineString", "coordinates": [[17.048037, 48.7292], [17.04821, 48.729427], [17.048768, 48.730076], [17.051731, 48.733064], [17.058085, 48.739292], [17.06103, 48.742093], [17.061823, 48.742758], [17.06263, 48.74338], [17.07355, 48.751425], [17.074463, 48.752141], [17.075166, 48.752836], [17.075949, 48.753529], [17.077102, 48.754903], [17.079066, 48.757244], [17.082493, 48.761463], [17.086979, 48.766871], [17.087415, 48.767362], [17.087878, 48.767784], [17.088296, 48.76809], [17.088752, 48.768383], [17.095509, 48.772692], [17.109721, 48.781717], [17.118059, 48.787046], [17.118766, 48.787461], [17.1221, 48.789607]

In [21]:
import json
import ast
import folium
from folium.features import CustomIcon, PolyLine


def generate_bridges_map(df_routes, up_until, zoom=15, fit=True, save=False):
    # Create a map centered around the first bridge's location
    first_bridge_location = [float(48.669),
                             float(19.699)]
    mymap = folium.Map(location=first_bridge_location,
                       zoom_start=zoom, tiles="Cartodb dark_matter")

    # Initialize a list to store all coordinates for fitting the map later
    all_coords = []

    for i in range(up_until):
        bridge = df_routes.iloc[i]

        # Extract coordinates for the bridge, starting node, and ending node
        bridge_location = [float(bridge['Zemepisná_šírka']), float(
            bridge['Zemepisná_dĺžka'])]
        starting_node_location = [
            float(bridge['StartNode_Latitude']), float(bridge['StartNode_Longitude'])]
        ending_node_location = [
            float(bridge['EndNode_Latitude']), float(bridge['EndNode_Longitude'])]

        # Add markers for the bridge, starting node, and ending node
            # Define the bridge marker icon
        bridge_icon = CustomIcon(
            icon_image="https://raw.githubusercontent.com/Medvedku/Assets/cee766454a39d9e86e58fb8d83965a6338c5de37/markers/bridge_marker_Cartodb.svg",
            icon_size=(50, 50),  # Adjust the size as needed
            icon_anchor=(25, 50),  # Adjust the anchor point as needed
        )

        # Define the marker icon
        start_marker_icon = CustomIcon(
            icon_image="https://raw.githubusercontent.com/Medvedku/Assets/main/markers/marker_Cartodb_start_end.svg",
            icon_size=(30, 30),  # Adjust the size as needed
            icon_anchor=(15, 30),  # Adjust the anchor point as needed
        )

        end_marker_icon = CustomIcon(
            icon_image="https://raw.githubusercontent.com/Medvedku/Assets/main/markers/marker_Cartodb_start_end.svg",
            icon_size=(30, 30),  # Adjust the size as needed
            icon_anchor=(15, 30),  # Adjust the anchor point as needed
        )

        # # Add a marker for the starting node
        # folium.Marker(starting_node_location, icon=start_marker_icon,
        #             tooltip='Starting Node').add_to(mymap)

        # # Add a marker for the ending node
        # folium.Marker(ending_node_location, icon=end_marker_icon,
        #             tooltip='Ending Node').add_to(mymap)

        # Add a marker for the bridge
        folium.Marker(bridge_location,
                    icon=bridge_icon,
                    tooltip=f'Bridge ({bridge["ID_mosta"]})').add_to(mymap)

        # Add the bridge, starting node, and ending node locations to all_coords
        all_coords.extend(
            [bridge_location, starting_node_location, ending_node_location])

        # Process and plot Car_route
        car_route_str = bridge['Car_route']
        if car_route_str:
            try:
                car_route_data = json.loads(bridge['Car_route'])
                car_route_coords = [(coord[1], coord[0]) for coord in car_route_data['paths'][0]['points']['coordinates']]
                PolyLine(car_route_coords, color="#ffffff",
                         weight=5, opacity=0.3).add_to(mymap)
            except:
                print(f"problem in {up_until}")

        # Process and plot Car_route
        car_route_alt_str = bridge['Car_alt']
        if car_route_alt_str:
            try:
                car_alt_route_data = json.loads(bridge['Car_alt'])
                car_alt_route_coords = [
                    (coord[1], coord[0]) for coord in car_alt_route_data['paths'][0]['points']['coordinates']]
                PolyLine(car_alt_route_coords, color="#ffffff",
                         weight=5).add_to(mymap)
            except:
                print(f"problem in {up_until}")


    # Fit the map to the bounds of all coordinates
    bounds = [list(map(min, zip(*all_coords))),
              list(map(max, zip(*all_coords)))]
    if fit:
        mymap.fit_bounds(bounds)
    if save:
        mymap.save("bridges_routes_network.html")


    return mymap

In [22]:
generate_bridges_map(df_routes=df_routes_sel, up_until=53, zoom=8, fit=False, save=True)

problem in 53
problem in 53
problem in 53
problem in 53
problem in 53
problem in 53
problem in 53
problem in 53
problem in 53


In [ ]:
def reduce_coordinates(coordinates):

    reduced_coordinates = [coordinates[0]]  # Keep the first item

    if len(coordinates) > 1:
        reduced_coordinates.append(coordinates[-1])  # Keep the last item

    # Keep every sixth item in between (starting from index 1)
    reduced_coordinates.extend(coordinates[1::6])

    return reduced_coordinates

In [ ]:
reduced_coords = reduce_coordinates(coordinates)
reduced_coords

[[18.880836, 48.777889],
 [18.90139, 48.75081],
 [18.880084, 48.778051],
 [18.877383, 48.779444],
 [18.875785, 48.781266],
 [18.87886, 48.793287],
 [18.879127, 48.804639],
 [18.880866, 48.809622],
 [18.878836, 48.816458],
 [18.874996, 48.830352],
 [18.874348, 48.833643],
 [18.872015, 48.836694],
 [18.870104, 48.839206],
 [18.867745, 48.840368],
 [18.864343, 48.843535],
 [18.857803, 48.843428],
 [18.852549, 48.84606],
 [18.847695, 48.84768],
 [18.843217, 48.852753],
 [18.840356, 48.857569],
 [18.838772, 48.860732],
 [18.838559, 48.86171],
 [18.837942, 48.863539],
 [18.837948, 48.865494],
 [18.841597, 48.869554],
 [18.840104, 48.874458],
 [18.806042, 48.865785],
 [18.797024, 48.863823],
 [18.794344, 48.862855],
 [18.789766, 48.867189],
 [18.783246, 48.870815],
 [18.778787, 48.870822],
 [18.772718, 48.870211],
 [18.766749, 48.870889],
 [18.764316, 48.870274],
 [18.759815, 48.871218],
 [18.758321, 48.871731],
 [18.757782, 48.872995],
 [18.753112, 48.874206],
 [18.748579, 48.873914],
 [18.7